# Projet Intermovie

Ce Notebook a pour but d'analyser un dataset de films dans le but de récupérer plusieurs informations :

- La liste des acteurs par film.

- La liste des films Américains (en gardant leur nom en français) et leur note moyenne.

- Les notes moyennes des différents genres.

- La note moyenne de chaque acteur par rapport aux films dans lesquels il apparaît.

## Structure des fichiers tsv

- name.basics :         nconst / primaryName / birthYear / deathYear / primaryProfession / knownForTitles
- title.akas :          titleId / ordering / title / region / language / types / attributes / isOriginalTitle
- title.basics :        tconst / titleType / primaryTitle / originalTitle / isAdult / startYear / endYear / runtimeMinutes / genres
- title.principals :    tconst / ordering / nconst / category / job / characters
- title.ratings :       tconst / averageRating / numVotes

## Nombre de lignes par échantillon

Nous allons utiliser pour chaque dataframe un maximum de 1000 lignes afin d'avoir un échantillon représentatif sans avoir de latence sur le traitement des données.


## 1- Liste des acteurs par film

In [3]:
# Extension IPython rechargeant les modules avant que l'utilisateur saisisse du code.
%load_ext autoreload
%autoreload 2

# Import des librairies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# On lit le fichier name.basics que l'on insère dans un dataframe.
df_noms = pd.read_table("./data/name.basics.tsv")

In [7]:
df_noms.columns


Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')

In [16]:
df_noms.drop(['birthYear', 'deathYear'], axis = 1)

In [18]:
df_noms.dropna(axis = 0)

,nconst,primaryName,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0043044"
1,nm0000002,Lauren Bacall,"actress,soundtrack","tt0071877,tt0117057,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,"actress,soundtrack,producer","tt0054452,tt0049189,tt0059956,tt0057345"
4,nm0000005,Ingmar Bergman,"writer,director,actor","tt0069467,tt0050976,tt0083922,tt0050986"
5,nm0000006,Ingrid Bergman,"actress,soundtrack,producer","tt0038109,tt0036855,tt0077711,tt0038787"
...,...,...,...,...
9706912,nm9993709,Lu Bevins,director,tt8744286
9706916,nm9993713,Sambit Mishra,writer,tt8325250
9706917,nm9993714,Romeo del Rosario,"animation_department,art_department",tt2455546
9706919,nm9993717,Harikrishnan Rajan,cinematographer,tt8736744


In [27]:
df_noms.drop_duplicates()

In [29]:
 df_noms[['primaryName', 'primaryProfession', 'knownForTitles']][df_noms['primaryProfession'].str.contains('actor|actress', regex = True)]

In [30]:
df_noms.describe()

,primaryName,primaryProfession,knownForTitles
count,3624969,3624969,3624969
unique,3172763,4980,1914197
top,Alex,actor,\N
freq,128,1772491,247943


In [ ]:
df_noms['knownForTitles'].apply(lambda x : pd.Series(x.split(',')))